In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\HAPPYUSER\\Documents\\Web_Development\\ML_AI_MLOPS\\Sports-image-classification'

In [20]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_random_rotation: float
    params_random_contrast: float
    params_random_translation_width: float
    params_random_translation_height: float

In [21]:
from SportsImageClassifier.constants import *
from SportsImageClassifier.utils.common import read_yaml, create_directories

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_random_rotation=self.params.RANDOM_ROTATION_FACTOR,
            params_random_contrast=self.params.RANDOM_CONTRAST_FACTOR,
            params_random_translation_width=self.params.RANDOM_TRANSLATION_WIDTH_FACTOR,
            params_random_translation_height=self.params.RANDOM_TRANSLATION_HEIGHT_FACTOR
        )

        return prepare_base_model_config

In [23]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [24]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.MobileNetV2(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights
            )

        self.save_model(path=self.config.base_model_path, model=self.model)


    def image_augmentation(self):
        img_augmentation = Sequential(
        [
            layers.RandomRotation(factor=self.config.params_random_rotation),
            layers.RandomTranslation(height_factor=self.config.params_random_translation_height, width_factor=self.config.params_random_translation_width),
            layers.RandomFlip(),
            layers.RandomContrast(factor=self.config.params_random_contrast),
        ],
        name="img_augmentation",
            )
        return img_augmentation


    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, img_augmentation):
        preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
        if freeze_all:
            model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:freeze_till]:
                layer.trainable = False
        
        global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
        prediction_layer = tf.keras.layers.Dense(classes)
        inputs = tf.keras.Input(shape=(256, 256, 3))
        x = img_augmentation(inputs)
        x = preprocess_input(x)
        x = model(x, training=False)
        x = global_average_layer(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        outputs = prediction_layer(x)
        full_model = tf.keras.Model(inputs, outputs)
        
        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
            img_augmentation=self.image_augmentation()
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [26]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-09-02 20:33:48,434: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-02 20:33:48,439: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 20:33:48,441: INFO: common: created directory at: artifacts]
[2024-09-02 20:33:48,443: INFO: common: created directory at: artifacts/prepare_base_model]


C:\Users\HAPPYUSER\AppData\Local\Temp\ipykernel_9072\231421299.py:7: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  self.model = tf.keras.applications.MobileNetV2(


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ img_augmentation (Sequential)   │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ true_divide_2 (TrueDivide)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ subtract_2 (Subtract)           │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 8, 8, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │       128,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,386,084 (9.10 MB)

 Trainable params: 128,100 (500.39 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

'artifacts/prepare_base_model'